# Chapter 2 Working with Data

In [5]:
from importlib.metadata import version

pkg = ['tiktoken','torch']
for p in pkg:
    print(f"{p} version: {version(p)}")

tiktoken version: 0.12.0
torch version: 2.3.0


In [11]:
import os 
import requests

if not os.path.exists("the-verdict.txt"):
    url = (
        "https://raw.githubusercontent.com/rasbt/"
        "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
        "the-verdict.txt"
    )
    file_path = "the-verdict.txt"

    response = requests.get(url, timeout=30)
    response.raise_for_status()
    with open(file_path, "wb") as f:
        f.write(response.content)

In [13]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [23]:
import re
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

In [25]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


In [27]:
vocab = {token:integer for integer,token in enumerate(all_words)}

In [29]:
class TokenizerV1:
    def __init__ (self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
        
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text
    

In [33]:
tokenizer = TokenizerV1(vocab)

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [37]:
decodedText = tokenizer.decode(ids)
print(decodedText)

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [39]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [41]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [43]:
#manages edge cases when the vocabulary doesnt include the word by using special tokens such as </unk>
class TokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

## DataLoaders

In [80]:
from torch.utils.data import Dataset, DataLoader
import tiktoken
import torch

class GPTDataSet(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids= []
        self.target_ids = []
        token_ids = tokenizer.encode(txt, allowed_special = {"<|endoftext|>"})

    # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0,len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1 : i+max_length +1]
            
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [82]:
def createDataLoader(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDataSet(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader
    

In [84]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [86]:
dataloader = createDataLoader(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)
second_batch = next(data_iter)
print(second_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [90]:
dataloader = createDataLoader(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


In [106]:
input_ids = torch.tensor([2, 3, 5, 1])

In [108]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [110]:
count = 0
for i in embedding_layer.weight:
    count+=1
    print(count, i)

1 tensor([ 0.3374, -0.1778, -0.1690], grad_fn=<UnbindBackward0>)
2 tensor([0.9178, 1.5810, 1.3010], grad_fn=<UnbindBackward0>)
3 tensor([ 1.2753, -0.2010, -0.1606], grad_fn=<UnbindBackward0>)
4 tensor([-0.4015,  0.9666, -1.1481], grad_fn=<UnbindBackward0>)
5 tensor([-1.1589,  0.3255, -0.6315], grad_fn=<UnbindBackward0>)
6 tensor([-2.8400, -0.7849, -1.4096], grad_fn=<UnbindBackward0>)


In [112]:
embedding_layer(input_ids)

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)

In [116]:
#now we use the full vocabulary used by the gpt2 model and configure a 256 dimensional vector
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [124]:
max_length = 4
dataloader = createDataLoader(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [126]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [130]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

torch.Size([8, 4, 256])


In [138]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [142]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings)

tensor([[[-0.7803,  0.4931,  0.6664,  ..., -0.2982,  0.2054, -1.1181],
         [-0.1678,  0.6988, -0.6209,  ...,  1.9709,  0.6565,  0.5605],
         [-0.0300, -1.1536,  0.5860,  ..., -1.3943, -0.8586,  1.5175],
         [-0.2712,  0.9240, -0.0620,  ...,  0.3604, -0.2292,  0.4807]],

        [[ 1.3713, -1.7873, -0.0424,  ...,  0.1224,  0.7864,  0.5912],
         [-0.5420,  1.0623, -0.6529,  ...,  1.1789,  0.7068,  2.2713],
         [-0.5060,  1.2555, -1.3363,  ..., -1.1436, -0.9601, -0.0268],
         [-0.4201,  1.8450, -1.0690,  ...,  1.9112, -1.5502,  0.7546]],

        [[-0.4767, -0.4201,  1.8582,  ...,  1.4673,  1.2550,  3.0997],
         [-0.2134, -0.4200, -0.8839,  ..., -0.3948,  0.8995,  2.0208],
         [ 1.8851, -1.4240,  0.0992,  ..., -0.8879, -1.2938,  0.9217],
         [-2.7090,  1.2139, -0.1789,  ...,  1.4028, -0.1912,  1.1062]],

        ...,

        [[-2.7588, -1.9927,  1.5728,  ...,  1.2705, -0.5514,  1.1914],
         [-0.5071, -0.4319,  0.0789,  ...,  0.4547, -0.21